# Importing csv file with coordinates

In [1]:
!pip install folium
import folium

In [2]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
df_data_1.shape

(103, 3)

# Scrapping Wikipedia

In [33]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [5]:
# GET request
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
# getting toronto postal codes data from soup tag
table_data = soup.find_all('table')[0]

In [7]:
# auxiliar list
table_content = []

# getting postal code, borough and neighborhoods for each data in previous table
for tdTag in table_data.find_all('td'):
    
    # removing posible start and ending spaces
    text = tdTag.text.strip()  
    
    # to ignore postal codes with no assign borough and neighborhoods
    if 'not assigned' in text.lower():
        continue
        
    # storing postal code, borough and neighborhoods
    postalCode = text[:3]
    textList = text[3:].split('(')
    borough = textList[0]
    neighborhoods = textList[1].strip()
    neighborhoods = neighborhoods[:neighborhoods.find(')')]
    neighborhoods = neighborhoods.replace(' /', ',') if ' /' in neighborhoods else neighborhoods
    
    # appending row
    table_content.append({'Postal Code': postalCode, 'Borough': borough, 'Neighborhood': neighborhoods})

# final dataframe
df = pd.DataFrame(table_content)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [8]:
print(df.shape)

(103, 3)


# Joining dataframes

In [9]:
data_merged = pd.merge(df, df_data_1, on='Postal Code', how='left', )
data_merged.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Visualizing Neighborhoods

In [10]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(data_merged['Latitude'], data_merged['Longitude'], data_merged['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# Using foursquare API

__Defining Credentials__
They have been hidden

In [11]:
# The code was removed by Watson Studio for sharing.

In [12]:
# create the API request URL
url = "https://api.foursquare.com/v3/places/nearby?ll={}%2C{}".format(43.709577, -79.445073)
headers = {"Accept": "application/json", "Authorization": APIKEY}

# make the GET request
results = requests.get(url, headers=headers).json()['results']

[{'id': 13276, 'name': 'Sushi Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_', 'suffix': '.png'}}]
[{'id': 13276, 'name': 'Sushi Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_', 'suffix': '.png'}}]
[{'id': 11062, 'name': 'Barbershop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/salon_barber_', 'suffix': '.png'}}]
[{'id': 17029, 'name': 'Convenience Store', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/conveniencestore_', 'suffix': '.png'}}]
[{'id': 17000, 'name': 'Retail', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_', 'suffix': '.png'}}]
[]
[{'id': 16032, 'name': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}}]
[{'id': 19046, 'name': 'Metro Station', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/subway_', 'suffix': '.png'}}]


In [13]:
# defining function to get Venues and Venues Categories for each neighborhood
def getNearbyVenues(names, latitudes, longitudes):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = "https://api.foursquare.com/v3/places/nearby?ll={}%2C{}".format(lat, lng)
        headers = {"Accept": "application/json", "Authorization": APIKEY}

        # make the GET request
        results = requests.get(url, headers=headers).json()['results']

        # return only relevant information for each nearby venue
        for v in results:
            
            # few vanues does not belongs to any category, so I prefered to ignore them, must of them do belong to some category
            if len(v['categories']) != 0:
                venues_list.append([(
                    name,
                    lat,
                    lng,
                    v['name'],
                    v['geocodes']['main']['latitude'],
                    v['geocodes']['main']['longitude'],
                    v['categories'][0]['name'])])

    items = [item for venue_list in venues_list for item in venue_list]
    nearby_venues = pd.DataFrame(items)
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude',
                             'Neighborhood Longitude', 'Venue',
                             'Venue Latitude', 'Venue Longitude',
                             'Venue Category']

    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(data_merged['Neighborhood'], data_merged['Latitude'], data_merged['Longitude'])
toronto_venues.head()

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Riverdale


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Bakery
2,Parkwoods,43.753259,-79.329656,CAPREIT Apartments,43.753180,-79.338970,Residential Building
3,Parkwoods,43.753259,-79.329656,CAPREIT Apartments,43.753920,-79.322400,Residential Building
4,Parkwoods,43.753259,-79.329656,8th Don Mills Cub Pack,43.748055,-79.336710,Other Great Outdoors


# Data Exploration before Clustering

Let's check how many venues were returned for each neighborhood

In [15]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,8,8,8,8,8,8
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",8,8,8,8,8,8
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",9,9,9,9,9,9
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",6,6,6,6,6,6
Woburn,5,5,5,5,5,5
Woodbine Heights,5,5,5,5,5,5


Let's find out how many unique categories can be curated from all the returned venues

In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,ATM,Accounting and Bookkeeping Service,Advertising Agency,African Restaurant,Airport,Alternative Medicine Clinic,American Restaurant,Art Gallery,Art Studio,...,Trade School,Travel Agency,Travel and Transportation,Turkish Restaurant,Used Car Dealership,Vegan and Vegetarian Restaurant,Veterinarian,Vintage and Thrift Store,Women's Store,Zoo
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Let's examine data size
toronto_onehot.shape

(752, 230)

In [19]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accounting and Bookkeeping Service,Advertising Agency,African Restaurant,Airport,Alternative Medicine Clinic,American Restaurant,Art Gallery,Art Studio,...,Trade School,Travel Agency,Travel and Transportation,Turkish Restaurant,Used Car Dealership,Vegan and Vegetarian Restaurant,Veterinarian,Vintage and Thrift Store,Women's Store,Zoo
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.142857,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
94,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.2,0.000000,0.0,0.0,0.0
95,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
96,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.142857,0.0,0.0,0.0


In [20]:
# confirming new size
toronto_grouped.shape

(98, 230)

In [21]:
# printing each neighborhood along with the top 5 most common venues
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                 Restaurant  0.25
1  Latin American Restaurant  0.12
2       Fast Food Restaurant  0.12
3             Clothing Store  0.12
4                Coffee Shop  0.12


----Alderwood, Long Branch----
                venue  freq
0        Dance Studio  0.14
1            Pizzeria  0.14
2        Fuel Station  0.14
3  Turkish Restaurant  0.14
4   Convenience Store  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                         venue  freq
0  Grocery Store / Supermarket  0.12
1          Fried Chicken Joint  0.12
2                         Park  0.12
3                   Restaurant  0.12
4         Fast Food Restaurant  0.12


----Bayview Village----
                    venue  freq
0            Hiking Trail  0.25
1                   Diner  0.25
2  Automotive Repair Shop  0.25
3      Chinese Restaurant  0.25
4                     ATM  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq

### 10 venues for each neighborhood

In [22]:
# auxiliary function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Restaurant,Latin American Restaurant,Fast Food Restaurant,Clothing Store,Coffee Shop,Rail Station,Indian Restaurant,Mexican Restaurant,Metro Station,Movie Theater
1,"Alderwood, Long Branch",Dance Studio,Pizzeria,Fuel Station,Turkish Restaurant,Convenience Store,Indian Restaurant,Restaurant,ATM,Middle Eastern Restaurant,Mobile Phone Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Grocery Store / Supermarket,Fried Chicken Joint,Park,Restaurant,Fast Food Restaurant,Middle Eastern Restaurant,Deli,Health and Medicine,Nursing Home,Night Club
3,Bayview Village,Hiking Trail,Diner,Automotive Repair Shop,Chinese Restaurant,ATM,Movie Theater,Medical Lab,Mediterranean Restaurant,Metals Supplier,Metro Station
4,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Dentist,Hair Salon,Italian Restaurant,Fast Food Restaurant,Pizzeria,Ice Cream Parlor,Jewish Restaurant,Middle School,Mobile Phone Store


# implementing cllustering (K means algorithm)

In [24]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(['Neighborhood'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 1, 2, 0, 4, 0, 1, 0, 3], dtype=int32)

Including labels found with model above

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = data_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

ValueError: cannot insert Cluster Labels, already exists

# Redrawing Neighborhood and their labels

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters